# Introduction

THis notebook contains he final notebook submission to the Kaggle competion predcit Calories challenge. I will start by first havoing code to the best possible solution based on the public leaderboard) and then explore other options. EDA and other stuff have been removed from this notebook for readability.

In [13]:
# imports

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from itertools import combinations
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from IPython.display import clear_output
import warnings
warnings.simplefilter('ignore')

In [7]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')

In [8]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']

def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

# Squares of features
train['dur_2']=train['Duration']*train['Duration']
test['dur_2']=test['Duration']*test['Duration']

train['hr_2']=train['Heart_Rate']*train['Heart_Rate']
test['hr_2']=test['Heart_Rate']*test['Heart_Rate']

train['height_2']=train['Height']*train['Height']
test['height_2']=test['Height']*test['Height']

train['weight_2']=train['Weight']*train['Weight']
test['weight_2']=test['Weight']*test['Weight']

# Square root of Duration
train['dur_sqrt'] = train['Duration'] ** 0.5
test['dur_sqrt'] = test['Duration'] ** 0.5

train['hr_sqrt'] = train['Heart_Rate'] ** 0.5
test['hr_sqrt'] = test['Heart_Rate'] ** 0.5

train['weight_sqrt'] = train['Weight'] ** 0.5
test['weight_sqrt'] = test['Weight'] ** 0.5

train['height_sqrt'] = train['Height'] ** 0.5
test['height_sqrt'] = test['Height'] ** 0.5

train['Intensity'] = train['Heart_Rate'] / train['Duration']
test['Intensity'] = test['Heart_Rate'] / test['Duration']

# Create binary indicators
train['is_male'] = (train['Sex'].str.lower() == 'male').astype(int)
train['is_female'] = (train['Sex'].str.lower() == 'female').astype(int)

test['is_male'] = (test['Sex'].str.lower() == 'male').astype(int)
test['is_female'] = (test['Sex'].str.lower() == 'female').astype(int)

# Numerical columns to interact with sex
num_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height', 'Body_Temp','bmi','Intensity','dur_body_heart']

# Create interaction features
for col in num_cols:
    train[f'{col}_Male'] = train[col] * train['is_male']
    train[f'{col}_Female'] = train[col] * train['is_female']

    test[f'{col}_Male'] = test[col] * test['is_male']
    test[f'{col}_Female'] = test[col] * test['is_female']


for col in ['Weight', 'Height', 'Duration', 'Heart_Rate', 'Body_Temp']:
    train[f'{col}_log'] = np.log1p(train[col])
    test[f'{col}_log'] = np.log1p(test[col])

train['Weight_Height_ratio'] = train['Weight'] / train['Height']
test['Weight_Height_ratio'] = test['Weight'] / test['Height']

train['HeartRate_BodyTemp_ratio'] = train['Heart_Rate'] / train['Body_Temp']
test['HeartRate_BodyTemp_ratio'] = test['Heart_Rate'] / test['Body_Temp']


train['effort_per_kg'] = train['Duration'] * train['Heart_Rate'] / train['Weight']
test['effort_per_kg'] = test['Duration'] * test['Heart_Rate'] / test['Weight']


train['Age_HeartRate'] = train['Age'] * train['Heart_Rate']
test['Age_HeartRate'] = test['Age'] * test['Heart_Rate']

train['Temp_Height'] = train['Body_Temp'] * train['Height']
test['Temp_Height'] = test['Body_Temp'] * test['Height']

from itertools import combinations

poly_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height']
for col1, col2 in combinations(poly_cols, 2):
    train[f'{col1}_x_{col2}'] = train[col1] * train[col2]
    test[f'{col1}_x_{col2}'] = test[col1] * test[col2]
'''
train['Duration_2'] = train['Duration']
test['Duration_2'] = test['Duration']

train['Age_2'] = train['Age']
test['Age_2'] = test['Age']
'''

"\ntrain['Duration_2'] = train['Duration']\ntest['Duration_2'] = test['Duration']\n\ntrain['Age_2'] = train['Age']\ntest['Age_2'] = test['Age']\n"

In [9]:
train.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories', 'bmi', 'dur_body_heart', 'BMR', 'dur_2',
       'hr_2', 'height_2', 'weight_2', 'dur_sqrt', 'hr_sqrt', 'weight_sqrt',
       'height_sqrt', 'Intensity', 'is_male', 'is_female', 'Age_Male',
       'Age_Female', 'Duration_Male', 'Duration_Female', 'Heart_Rate_Male',
       'Heart_Rate_Female', 'Weight_Male', 'Weight_Female', 'Height_Male',
       'Height_Female', 'Body_Temp_Male', 'Body_Temp_Female', 'bmi_Male',
       'bmi_Female', 'Intensity_Male', 'Intensity_Female',
       'dur_body_heart_Male', 'dur_body_heart_Female', 'Weight_log',
       'Height_log', 'Duration_log', 'Heart_Rate_log', 'Body_Temp_log',
       'Weight_Height_ratio', 'HeartRate_BodyTemp_ratio', 'effort_per_kg',
       'Age_HeartRate', 'Temp_Height', 'Age_x_Duration', 'Age_x_Heart_Rate',
       'Age_x_Weight', 'Age_x_Height', 'Duration_x_Heart_Rate',
       'Duration_x_Weight', 'Duration_x_Height', 'Heart_Rate_x_We

In [10]:
def rmsle_loss(y_pred, y_true):
    y_pred = torch.clamp(y_pred, min=0)
    y_true = torch.clamp(y_true, min=0)
    return torch.sqrt(F.mse_loss(torch.log1p(y_pred), torch.log1p(y_true)))

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [18]:
class CalorieNet(nn.Module):
    def __init__(self, input_dim, dropout=0.1, use_residual=True):
            super(CalorieNet, self).__init__()
            
            self.use_residual = use_residual
            
            self.input_norm = nn.BatchNorm1d(input_dim)
            self.backbone = nn.ModuleList([
                self._make_block(input_dim, 512, dropout),
                self._make_block(512, 256, dropout),
                self._make_block(256, 128, dropout),
                self._make_block(128, 64, dropout),
            ])
            
            self.output_head = nn.Sequential(
                nn.Linear(64, 32),
                nn.GELU(),
                nn.Dropout(dropout * 0.5),  
                nn.Linear(32, 1),
                nn.Softplus()
            )
            

            self._initialize_weights()
        
    def _make_block(self, in_features, out_features, dropout):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.GELU(),
            nn.Dropout(dropout)
        )
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.input_norm(x)
        for block in self.backbone:
            if self.use_residual and x.shape[1] == block[0].in_features and x.shape[1] == block[0].out_features:
                x = x + block(x)
            else:
                x = block(x)
        x = self.output_head(x)
        return x

In [25]:
drop_cols = ['id', 'Calories']
train['Sex'] = train['Sex'].map({'male': 1, 'female': 0})
test['Sex'] = test['Sex'].map({'male': 1, 'female': 0})
X = train.drop(columns=drop_cols)
y = train['Calories']


def get_max_min(series: pd.Series) -> list:
    return [series.max(), series.min()]

def max_min_scalse(series: pd.Series) -> pd.Series:
    max_min = get_max_min(series)
    return series.apply(lambda x: (x - max_min[1]) / (max_min[0] - max_min[1]))

for col in X.columns:
    if col != 'Sex':
        X[col] = max_min_scalse(X[col])

In [26]:

batch_size = 512

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [27]:
device

device(type='cpu')

In [29]:
X_train.shape

(600000, 59)

In [ ]:
model = CalorieNet(
        input_dim=59,  
        dropout=0.00,
        use_residual=True
    ).to(device)
no_decay = ['bias', 'LayerNorm.weight', 'BatchNorm']
decay_params = []
no_decay_params = []

for name, param in model.named_parameters():
    if not param.requires_grad:
        continue
    if any(nd in name for nd in no_decay):
        no_decay_params.append(param)
    else:
        decay_params.append(param)
        
optimizer_params = [
    {'params': decay_params, 'weight_decay': 5e-4}, 
    {'params': no_decay_params, 'weight_decay': 0.0}
]
loss_fn_rmsle = rmsle_loss
best_val_loss = float("inf")
patience = 25
counter = 0
scaler = torch.amp.GradScaler() if torch.cuda.is_available() else None
max_epochs = 500

optimizer = torch.optim.AdamW(
    optimizer_params,
    lr=3e-4, 
    betas=(0.9, 0.999),
    eps=1e-8,
    amsgrad=True
)

total_steps = max_epochs * len(train_loader)




scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-3,
    epochs=max_epochs,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,  
    anneal_strategy='cos',
    div_factor=10.0,
    final_div_factor=100.0
)

for epoch in range(max_epochs):
    model.train()
    total_train_loss = 0
    total_train_rmsle = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{max_epochs}")

    for batch_X, batch_y in progress_bar:
        batch_X = batch_X.to(device, non_blocking=True)
        batch_y = batch_y.to(device, non_blocking=True)

        if scaler:
            with torch.amp.autocast('cuda'):
                output = model(batch_X)
                loss = rmsle_loss(output, batch_y)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(batch_X)
            loss = rmsle_loss(output, batch_y)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()

        scheduler.step()  

        
        with torch.no_grad():
            rmsle = loss_fn_rmsle(output, batch_y)
            total_train_rmsle += rmsle.item()

        total_train_loss += loss.item()

        progress_bar.set_postfix(
            loss=loss.item(),
            rmsle=rmsle.item(),
            lr=scheduler.get_last_lr()[0]
        )

    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_rmsle = total_train_rmsle / len(train_loader)

    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device, non_blocking=True)
            batch_y = batch_y.to(device, non_blocking=True)

            output = model(batch_X)
            val_loss = loss_fn_rmsle(output, batch_y)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    print(f"\nEpoch {epoch+1}/{max_epochs}")
    print(f"Train Loss: {avg_train_loss:.6f}, Train RMSLE: {avg_train_rmsle:.6f}")
    print(f"Val RMSLE: {avg_val_loss:.6f}, LR: {scheduler.get_last_lr()[0]:.8f}")

    if avg_val_loss < best_val_loss:
        improvement = (best_val_loss - avg_val_loss) / best_val_loss * 100
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(model.state_dict(), "calorie_model_best.pt")
        clear_output(wait=True)
        print(f"Saved best model (RMSLE: {best_val_loss:.6f}, improved by {improvement:.2f}%)")
        
    else:
        counter += 1
        print(f"No improvement for {counter}/{patience} epochs")
        if counter >= patience:
            
            print(f"Early stopping triggered after {patience} epochs without improvement")
            print(f"Best RMSLE: {best_val_loss:.6f}")
            break

Saved best model (RMSLE: 0.059300, improved by 0.33%)


Epoch 54/500: 100%|██████████████████████████████████████| 1172/1172 [00:09<00:00, 125.55it/s, loss=0.0668, lr=0.003, rmsle=0.0668]



Epoch 54/500
Train Loss: 0.062202, Train RMSLE: 0.062202
Val RMSLE: 0.060751, LR: 0.00299942
No improvement for 1/25 epochs


Epoch 55/500: 100%|██████████████████████████████████████| 1172/1172 [00:08<00:00, 138.14it/s, loss=0.0626, lr=0.003, rmsle=0.0626]



Epoch 55/500
Train Loss: 0.061876, Train RMSLE: 0.061876
Val RMSLE: 0.059743, LR: 0.00299909
No improvement for 2/25 epochs


Epoch 56/500: 100%|██████████████████████████████████████| 1172/1172 [00:05<00:00, 201.89it/s, loss=0.0545, lr=0.003, rmsle=0.0545]



Epoch 56/500
Train Loss: 0.062103, Train RMSLE: 0.062103
Val RMSLE: 0.060485, LR: 0.00299869
No improvement for 3/25 epochs


Epoch 57/500:  47%|██████████████████▎                    | 549/1172 [00:03<00:02, 211.28it/s, loss=0.0861, lr=0.003, rmsle=0.0861]

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import pandas as pd

# Setup
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex','Duration','Age']
categorical_cols = ['Sex','Duration']#,'Age']
# Stratification based on binned Duration
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Stratified K-Fold
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

# Log-transform the target
y_log = np.log1p(y)

# Initialize predictions
y_pred = np.zeros(len(test))
oof_preds = np.zeros(len(train))  # To store OOF predictions

for idx_fold, (idx_train, idx_valid) in enumerate(cv.split(X, duration_bins)):
    print(f"\nFold {idx_fold + 1}")
    
    X_train, y_train_fold = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid_fold = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoder = TargetEncoder(random_state=42)
    X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols], y_train_fold)
    X_valid[categorical_cols] = encoder.transform(X_valid[categorical_cols])
    X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])

    # Model
    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train_fold,
        eval_set=[(X_valid, y_valid_fold)],
        verbose=100
    )

    # Predict on test and validation sets
    y_pred += model.predict(X_test)
    oof_preds[idx_valid] = np.expm1(model.predict(X_valid))

# Final test prediction (average)
y_pred = np.expm1(y_pred / cv.get_n_splits()) 

# OOF RMSLE
rmsle = np.sqrt(mean_squared_error(np.log1p(train['Calories']), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle:.5f}")

# Submission
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': y_pred})
submission_xgb.to_csv('submission_xgb_category.csv', index=False)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
# Setup
X_train = X
y_train = y
X_test = test.drop(columns=['id'])
categorical_cols = ['Sex']#,'Duration','Age']

# Stratify on Duration bins
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize prediction arrays
test_preds_log = np.zeros(len(test))       # accumulate raw predictions (log)
test_preds_lin = np.zeros(len(test))       # accumulate converted predictions
oof_preds = np.zeros(len(train))           # OOF predictions for validation

#X_train['Duration'] = X_train['Duration'].astype(int)
#X_test['Duration'] = X_test['Duration'].astype(int)
# Cross-validation loop
for fold, (idx_tr, idx_val) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_tr], X_train.iloc[idx_val]
    y_tr, y_val = np.log1p(y_train.iloc[idx_tr]), np.log1p(y_train.iloc[idx_val])
    
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_cols)
    val_pool = Pool(X_val, y_val, cat_features=categorical_cols)
    test_pool = Pool(X_test, cat_features=categorical_cols)


    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test
    preds_test_log = model.predict(test_pool)
    preds_test_lin = np.expm1(preds_test_log)

    test_preds_log += preds_test_log
    test_preds_lin += preds_test_lin

    # OOF predictions for this fold
    oof_preds[idx_val] = np.expm1(model.predict(val_pool))

# Average predictions
test_preds_log = np.expm1(test_preds_log / cv.get_n_splits())  # average in log space
test_preds_lin = test_preds_lin / cv.get_n_splits()            # average in linear space

# Compute OOF RMSLE
rmsle_oof = np.sqrt(mean_squared_error(np.log1p(y_train), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle_oof:.5f}")

# Save predictions
test['Calories'] = test_preds_lin
test[['id', 'Calories']].to_csv('catboost_linear_avg_new_age.csv', index=False)

test['Calories'] = test_preds_log
test[['id', 'Calories']].to_csv('catboost_log_avg_new_age.csv', index=False)

In [ ]:
X

In [ ]:
# Save predictions
test['Calories'] = test_preds_lin
test[['id', 'Calories']].to_csv('catboost_linear_avg_age_cat.csv', index=False)

test['Calories'] = test_preds_log
test[['id', 'Calories']].to_csv('catboost_log_avg_age_cat.csv', index=False)

In [ ]:
X_test[categorical_cols].value_counts()

In [ ]:
test.head()

# Ensemble

In [ ]:
data1=pd.read_csv('submission_xgb_new (1).csv')
data2=pd.read_csv('catboost_log_avg (2).csv')

data = data1.merge(data2, on='id', suffixes=('_xgb', '_cat'))


# 1. Simple average of the two predictions
data['Calories_avg'] = (data['Calories_xgb'] + data['Calories_cat']) / 2

# 2. Log-scale average (RMSLE-style averaging)
data['Calories_log_avg'] = np.expm1(
    (np.log1p(data['Calories_xgb']) + np.log1p(data['Calories_cat'])) / 2
)

print(data[['id', 'Calories_xgb', 'Calories_cat', 'Calories_avg', 'Calories_log_avg']].head())
data[['id', 'Calories_avg']].rename(columns={'Calories_avg': 'Calories'}).to_csv('submission_reg_avg_2.csv', index=False)

# Save log-scale average with 'Calories' as the second column
data[['id', 'Calories_log_avg']].rename(columns={'Calories_log_avg': 'Calories'}).to_csv('submission_log_avg_2.csv', index=False)